In [5]:
model_name = "Support Vector Classifier"
model_version = 1

In [6]:
import mlflow.pyfunc

model_version = 1

model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}/{model_version}"
)

MlflowException: Could not find registered model with name Support Vector Classifier